In [ ]:

!pip -q install PyPDF2==3.0.1 transformers==4.41.2 datasets==2.19.0 torch torchvision torchaudio --upgrade --quiet
!pip -q install networkx==3.3 scikit-learn==1.5.1 sentencepiece==0.2.0 tabulate==0.9.0 --quiet
!pip -q install TTS==0.22.0 --quiet
!pip -q install moviepy==1.0.3 --quiet
print('✅ Installed')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 955.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 60

In [ ]:

import re, os, math, time, uuid
from typing import List, Dict
from pathlib import Path
from PyPDF2 import PdfReader
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
from moviepy.editor import TextClip, AudioFileClip, concatenate_videoclips
from IPython.display import Audio
print('✅ Imported')


In [ ]:

from google.colab import files
uploaded = files.upload()  # choose your research paper PDF
PDF_PATH = list(uploaded.keys())[0]
print('Using:', PDF_PATH)


In [ ]:

SECTION_PATTERNS = [
    r'^\s*abstract\s*$', r'^\s*introduction\s*$', r'^\s*related work\s*$',
    r'^\s*methods?\s*$', r'^\s*materials and methods\s*$', r'^\s*experiments?\s*$',
    r'^\s*results?\s*$', r'^\s*discussion\s*$', r'^\s*conclusion\s*$', r'^\s*references?\s*$'
]
SECTION_RE = re.compile("|".join(SECTION_PATTERNS), flags=re.IGNORECASE)

def read_pdf_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text_chunks = []
    for page in reader.pages:
        txt = page.extract_text() or ""
        text_chunks.append(txt)
    return "\n".join(text_chunks)

def normalize(text: str) -> str:
    return re.sub(r'\s+', ' ', text).strip()

def split_into_paragraphs(text: str, min_len: int = 40):
    candidates = re.split(r'\n{2,}|(?<=\.)\s+', text)
    return [normalize(p) for p in candidates if len(normalize(p)) >= min_len]

def label_section_headers(paragraphs):
    labeled, current = [], "unknown"
    for p in paragraphs:
        if SECTION_RE.match(p.lower()):
            current = p.strip().lower()
            labeled.append({"text": p, "section_guess": "header"})
        else:
            labeled.append({"text": p, "section_guess": current})
    return labeled

raw_text = read_pdf_text(PDF_PATH)
paragraphs = split_into_paragraphs(raw_text)
labeled = label_section_headers(paragraphs)
print('Paragraphs:', len(paragraphs))
labeled[:3]


In [ ]:

zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
CANDIDATE_LABELS = ["abstract","introduction","methods","results","discussion","conclusion","other"]

def tag_paragraphs_zero_shot(paragraphs):
    tagged = []
    for p in paragraphs:
        res = zero_shot(p, CANDIDATE_LABELS, multi_label=False)
        tagged.append({"text": p, "section": res["labels"][0], "score": float(res["scores"][0])})
    return tagged


tagged = tag_paragraphs_zero_shot(paragraphs[:120])
tagged[:5]


In [ ]:

def split_sentences(text: str):
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]

def textrank_summarize(text: str, top_k: int = 5):
    sentences = split_sentences(text)
    if len(sentences) <= top_k:
        return " ".join(sentences)
    vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words="english").fit(sentences)
    X = vectorizer.transform(sentences)
    sim = (X * X.T).toarray()
    for i in range(sim.shape[0]):
        sim[i, i] = 0.0
    nx_graph = nx.from_numpy_array(sim)
    scores = nx.pagerank(nx_graph, max_iter=100, tol=1e-6)
    ranked = sorted(((scores[i], s, i) for i, s in enumerate(sentences)), reverse=True)
    chosen = sorted(ranked[:top_k], key=lambda x: x[2])
    return " ".join([c[1] for c in chosen])

def concat_section(tagged, target):
    return " ".join([t["text"] for t in tagged if t.get("section") == target])

abstract_txt = concat_section(tagged, "abstract")
methods_txt  = concat_section(tagged, "methods")
results_txt  = concat_section(tagged, "results")

narration_parts = []
if abstract_txt: narration_parts.append("Abstract: " + textrank_summarize(abstract_txt, top_k=2))
if methods_txt:  narration_parts.append("Methods: " + textrank_summarize(methods_txt, top_k=3))
if results_txt:  narration_parts.append("Results: " + textrank_summarize(results_txt, top_k=3))

narration_text = " ".join(narration_parts) if narration_parts else "Summary not available."
narration_text


In [ ]:

from TTS.api import TTS
MODEL_NAME = "tts_models/en/ljspeech/tacotron2-DDC"
OUT_WAV = "narration.wav"

tts = TTS(model_name=MODEL_NAME, progress_bar=True, gpu=True)
tts.tts_to_file(text=narration_text, file_path=OUT_WAV)
Audio(OUT_WAV)


In [ ]:

def text_slide(text, duration=5, width=1280, height=720):
    return (TextClip(text, fontsize=48, size=(width, height), method="caption")
            .set_duration(duration).set_position("center"))

slides_text = ["Paper Summary", "Methods Overview", "Key Findings"]
clips = [text_slide(t, duration=max(4, min(10, len(narration_text)//50))) for t in slides_text]
video = concatenate_videoclips(clips, method="compose")

audio = AudioFileClip(OUT_WAV)
final = video.set_audio(audio).set_duration(max(video.duration, audio.duration))
final.write_videofile("paper_summary.mp4", fps=24)
print('Saved: paper_summary.mp4')
